## Importing packages and defining functions 

In [1]:
from dask.distributed import LocalCluster, Client
import glob
import scipy.ndimage as ndimage
from scipy.ndimage.measurements import label, find_objects
import xarray as xr

In [2]:
local_dir = "/g/data/e14/cp3790/dask-workers"
cluster = LocalCluster(processes=False, local_dir=local_dir)
client = Client(cluster)
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.10/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.10/lib/python3.6/site-packages/distributed/worker.py:485: UserWarning: The local_dir keyword has moved to local_directory
  warnings.warn("The local_dir keyword has moved to local_directory")


Client Scheduler: inproc://10.0.64.14/27559/1 Dashboard: http://10.0.64.14/27559/1:42292/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


In [3]:
def consec(temps):
    labels, _ = label(temps) # labels the occurrence of non-zero values (in this case, 1) and gives it an 'event' number 
    slices = find_objects(labels) 
    new_temps = np.zeros(len(temps))
    
    for i in slices:
        if temps[i].size >= 3:
            new_temps[i] = temps[i]
        else:
            new_temps[i] = 0
    return new_temps

In [4]:
def consec_ufunc(array, axis):
    result = np.apply_along_axis(
        consec, 
        axis,
        array
    )
    return result 

In [5]:
def consecutive(dataarray):
    result = xr.apply_ufunc(
        consec_ufunc, dataarray,
        input_core_dims=[['time']],
        output_core_dims=[['time']],
        kwargs={'axis':-1},
        output_sizes={'time':len(dataarray['time'])}
    )
    return result.transpose(*dataarray.dims)

## Opening files

In [6]:
mean_climatology = xr.open_dataarray('/g/data/e14/cp3790/Charuni/climatology-australia.nc')
threshold = xr.open_dataarray('/g/data/e14/cp3790/Charuni/threshold-australia.nc')

In [7]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/ERA5-new/era5_dailytmax_*.nc'))

obs_aus = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1982', '2018'), longitude=slice(113, 154), latitude=slice(-10, -44))

obs_aus_tmax = obs_aus["dmax"].load() #.sel(time=~((era5_dailytmax_aus["dmax"].time.dt.month == 2) & (era5_dailytmax_aus["dmax"].time.dt.day == 29)))
obs_aus_tmax.attrs['units'] = 'deg C'

## Calculations 

In [8]:
threshold_anomaly = threshold - mean_climatology

In [9]:
dailytmax_anomaly = obs_aus_tmax.groupby('time.dayofyear') - mean_climatology 

In [10]:
thw_severity = (dailytmax_anomaly.groupby('time.dayofyear'))/threshold_anomaly

In [11]:
thw_severity

<xarray.DataArray (time: 13514, latitude: 137, longitude: 165)>
array([[[-6.84762382e-01, -4.59847106e-01, -2.82294214e-01, ...,
          5.10622368e-01,  3.10134019e-01, -1.14468442e-02],
        [-3.02013844e-01, -2.25452811e-01, -2.00023624e-01, ...,
          1.01169290e-01, -2.49706942e-02, -2.18906334e-01],
        [-8.06197849e-02, -5.39955877e-02, -1.32506473e-01, ...,
         -8.23025786e-01, -7.23368038e-01, -6.20517455e-01],
        ...,
        [-4.85689756e-01, -5.01525366e-01, -5.12991905e-01, ...,
          3.48985278e-01,  3.73230936e-01,  4.01076640e-01],
        [-4.06101271e-01, -4.28536852e-01, -4.42150665e-01, ...,
          3.85285060e-01,  3.87325818e-01,  4.08718199e-01],
        [-3.34978690e-01, -3.54140322e-01, -3.61609388e-01, ...,
          4.11954020e-01,  4.06475354e-01,  4.34536847e-01]],

       [[-3.13019704e-01, -2.49810278e-01, -1.36460613e-01, ...,
         -3.29109461e-02,  9.33223390e-02, -5.41669747e-02],
        [-2.42801736e-01, -1.49368606e-01, -3.31535745e-02, ...,
         -1.80585431e-02, -3.81956246e-02, -4.53739553e-02],
        [-7.87760103e-02, -2.94321554e-02,  2.39704018e-02, ...,
         -4.36406856e-01, -4.49558465e-01, -3.20848725e-01],
        ...,
        [-1.10074937e+00, -1.06346111e+00, -1.02704317e+00, ...,
          1.42633840e-01,  1.51393068e-01,  1.50437158e-01],
        [-1.16132321e+00, -1.13500927e+00, -1.10579046e+00, ...,
          2.14801566e-01,  1.82550630e-01,  1.76959761e-01],
        [-1.24186787e+00, -1.21439744e+00, -1.16890596e+00, ...,
          2.17040913e-01,  2.16441145e-01,  1.79150118e-01]],

       [[ 4.35896234e-01,  5.99071295e-01,  5.89050903e-01, ...,
         -6.01907381e-01, -4.79133897e-01, -3.26400097e-01],
        [ 2.71258947e-01,  2.46002712e-01,  2.47372091e-01, ...,
         -3.11283768e-01, -3.03393523e-01, -3.73146182e-01],
        [ 8.37475466e-02,  1.82640721e-01,  2.28476321e-01, ...,
         -2.14668291e-01, -2.35647536e-01, -3.43020433e-01],
        ...,
        [-2.95618765e-01, -3.25730836e-01, -3.51618427e-01, ...,
          2.14617084e-01,  2.17850279e-01,  1.87573359e-01],
        [-3.25864968e-01, -3.13133120e-01, -3.14241532e-01, ...,
          2.11298276e-01,  2.56200139e-01,  2.63472037e-01],
        [-2.54815509e-01, -3.03519263e-01, -2.88499823e-01, ...,
          6.51472327e-02,  2.00706506e-01,  2.28675704e-01]],

       ...,

       [[ 2.36733093e-01,  8.14368879e-02, -7.33130308e-02, ...,
         -4.14150578e-02, -1.87609862e-01, -4.23360902e-01],
        [ 1.91333750e-01,  6.42541549e-02, -1.05526709e-01, ...,
         -2.12197442e-01, -4.46687942e-01, -7.10977298e-01],
        [-1.07226034e-02, -9.81667768e-02, -1.81278640e-01, ...,
         -1.30756437e-01, -3.00982415e-01, -4.14879779e-01],
        ...,
        [-4.85358387e-01, -4.71332556e-01, -4.66185105e-01, ...,
          2.15888118e+00,  1.96836124e+00,  1.95932287e+00],
        [-5.37795632e-01, -5.26178190e-01, -5.08088007e-01, ...,
          2.24766379e+00,  2.19827163e+00,  2.17088104e+00],
        [-5.95772413e-01, -5.88390167e-01, -5.63174348e-01, ...,
          2.27353349e+00,  2.26015459e+00,  2.29403874e+00]],

       [[-1.89891772e-01, -9.67123427e-02, -1.84710924e-03, ...,
          3.99708509e-01,  5.69619432e-01,  6.32553440e-01],
        [-1.78161734e-01, -1.54209095e-01, -1.09180627e-01, ...,
          7.43222939e-01,  9.78949424e-01,  1.03761379e+00],
        [-2.23405740e-01, -2.01212909e-01, -1.45469813e-01, ...,
          7.42025818e-01,  9.96551173e-01,  1.16929158e+00],
        ...,
        [-4.03448430e-01, -4.26545484e-01, -4.18795085e-01, ...,
          4.61542650e-01,  5.39570392e-01,  6.59541923e-01],
        [-4.07352455e-01, -4.27655930e-01, -4.39281946e-01, ...,
          3.99514097e-01,  4.22718559e-01,  5.04460630e-01],
        [-3.66143843e-01, -3.92741847e-01, -4.02914304e-01, ...,
          4.25405195e-01,  4.29013673e-01,  4.62634498e-01]],

       [[-2.43763585e-02, -5.29499546e-02, -1.23664185e-02, ...,
   

In [15]:
#thw_severity = thw_severity.to_dataset(name='severity')

In [12]:
# Finding potential THW events
hws_preDur = xr.where(thw_severity > 0, thw_severity, 0)

In [13]:
hws_preDur

<xarray.DataArray (time: 13514, latitude: 137, longitude: 165)>
array([[[0.        , 0.        , 0.        , ..., 0.51062237,
         0.31013402, 0.        ],
        [0.        , 0.        , 0.        , ..., 0.10116929,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.34898528,
         0.37323094, 0.40107664],
        [0.        , 0.        , 0.        , ..., 0.38528506,
         0.38732582, 0.4087182 ],
        [0.        , 0.        , 0.        , ..., 0.41195402,
         0.40647535, 0.43453685]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.09332234, 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.0239704 , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.14263384,
         0.15139307, 0.15043716],
        [0.        , 0.        , 0.        , ..., 0.21480157,
         0.18255063, 0.17695976],
        [0.        , 0.        , 0.        , ..., 0.21704091,
         0.21644115, 0.17915012]],

       [[0.43589623, 0.59907129, 0.5890509 , ..., 0.        ,
         0.        , 0.        ],
        [0.27125895, 0.24600271, 0.24737209, ..., 0.        ,
         0.        , 0.        ],
        [0.08374755, 0.18264072, 0.22847632, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.21461708,
         0.21785028, 0.18757336],
        [0.        , 0.        , 0.        , ..., 0.21129828,
         0.25620014, 0.26347204],
        [0.        , 0.        , 0.        , ..., 0.06514723,
         0.20070651, 0.2286757 ]],

       ...,

       [[0.23673309, 0.08143689, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.19133375, 0.06425415, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 2.15888118,
         1.96836124, 1.95932287],
        [0.        , 0.        , 0.        , ..., 2.24766379,
         2.19827163, 2.17088104],
        [0.        , 0.        , 0.        , ..., 2.27353349,
         2.26015459, 2.29403874]],

       [[0.        , 0.        , 0.        , ..., 0.39970851,
         0.56961943, 0.63255344],
        [0.        , 0.        , 0.        , ..., 0.74322294,
         0.97894942, 1.03761379],
        [0.        , 0.        , 0.        , ..., 0.74202582,
         0.99655117, 1.16929158],
        ...,
        [0.        , 0.        , 0.        , ..., 0.46154265,
         0.53957039, 0.65954192],
        [0.        , 0.        , 0.        , ..., 0.3995141 ,
         0.42271856, 0.50446063],
        [0.        , 0.        , 0.        , ..., 0.42540519,
         0.42901367, 0.4626345 ]],

       [[0.        , 0.        , 0.        , ..., 0.83325015,
         0.9372029 , 0.83152614],
        [0.01891481, 0.        , 0.        , ..., 1.20552759,
         1.27917767, 1.21616395],
        [0.        , 0.        , 0.        , ..., 1.13185928,
         1.17168047, 1.1994832 ],
        ...,
        [0.        , 0.        , 0.        , ..., 1.31046593,
         1.31064992, 1.2820955 ],
        [0.        , 0.        , 0.        , ..., 1.27095228,
         1.2767429 , 1.29197314],
        [0.        , 0.        , 0.        , ..., 1.23392298,
         1.21912382, 1.26015814]]])
Coordinates:
  * longitude  (longitude) float32 113.0 113.25 113.5 ... 153.5 153.75 154.0
  * latitude   (latitude) float32 -10.0 -10.25 -10.5 ... -43.5 -43.75 -44.0
  * time       (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2018-12-31
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365

In [14]:
oscar = consecutive(hws_preDur)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

KeyboardInterrupt: 

In [23]:
hws_preDur.to_netcdf('/g/data/e14/cp3790/Charuni/hws_preDur.nc')